# Lets place this notebook in the root directory

In [1]:
# sys.path.insert(0,'..')

In [1]:
import os
path = %pwd
if path.split(os.sep)[-1] == 'best_model_conversion':
    %cd ..

/mnt/c/Users/Mustapha/Documents/competitions/klee_project_audio


Lets also refresh all our dependecies in run time

In [2]:
%reload_ext autoreload
%autoreload 2

load environment variables, if they exist

In [3]:
from dotenv import load_dotenv

load_dotenv(".env_consts")

True

# Import

In [4]:
# ----------------------------- Torch and friends ---------------------------- #
import torch
import onnx
import numpy as np

# ----------------------------------- other ---------------------------------- #
import os
from tqdm import tqdm
import wandb

# ---------------------------------- Custom ---------------------------------- #
from src.load_dataset_fft_aug import get_splitter_dataloaders_fft

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


salade de tensorflow

In [5]:
import tensorflow as tf

# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')

2022-03-10 14:51:41.985796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-10 14:51:42.022304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-10 14:51:42.022796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


# Utils for import

In [6]:
class FCRNNPermuteForLSTM(torch.nn.Module):
    """
    Permute the input from (batch, channel, time) to (batch, time, channel)
    """
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return x.permute(0, 2, 1)
    
class SequentialLSTM(torch.nn.Module):
    """
    LSTMs in Pytorch, outputs a tuple (output, (h_n, c_n)), we only need output
    """
    def __init__(self, input_size=1280, hidden_size=40, num_layers=1, dropout=0):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout)
    def forward(self, x):
        x, _ = self.lstm(x)
        return x

# Pytorch model to ONNX

### load model

In [7]:
model_loaded = torch.load("custom_models/F-CRNN_mae_49_58.pth")

In [8]:
model_loaded(torch.rand(1, 201, 498).to("cuda"))

tensor([[0.1424, 0.2618]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [9]:
model_loaded = torch.nn.Sequential( #input size = 80000
        torch.nn.Conv1d(201, 32, kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.MaxPool1d(kernel_size=2, stride=2),
        
        torch.nn.Conv1d(32, 64, kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.MaxPool1d(kernel_size=2, stride=2),
        
        torch.nn.Conv1d(64, 64, kernel_size=3),
        torch.nn.ReLU(),
        torch.nn.Dropout(0.5),
        
        FCRNNPermuteForLSTM(),
        torch.nn.LSTM(64, 40, 1, dropout=0.5),).to("cuda")

/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


### to onnx

In [10]:
onnx_output_folder = "best_model_conversion/onnx"
if not os.path.exists(onnx_output_folder):
    os.makedirs(onnx_output_folder)
torch.onnx.export(
    model_loaded,
    torch.randn(1, 201, 498, device="cuda"),
    os.path.join(onnx_output_folder, "fcrnn.onnx"),
    opset_version=12,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    # dynamic_axes={'input': {0: 'sequence'}, 'output': {0: 'sequence'}}
    # keep_initializers_as_inputs=True,
)

graph(%input : Float(1, 201, 498, strides=[100098, 498, 1], requires_grad=0, device=cuda:0),
      %0.weight : Float(32, 201, 3, strides=[603, 3, 1], requires_grad=1, device=cuda:0),
      %0.bias : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %3.weight : Float(64, 32, 3, strides=[96, 3, 1], requires_grad=1, device=cuda:0),
      %3.bias : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %6.weight : Float(64, 64, 3, strides=[192, 3, 1], requires_grad=1, device=cuda:0),
      %6.bias : Float(64, strides=[1], requires_grad=1, device=cuda:0),
      %118 : Float(1, 160, 64, strides=[10240, 64, 1], requires_grad=0, device=cuda:0),
      %119 : Float(1, 160, 40, strides=[6400, 40, 1], requires_grad=0, device=cuda:0),
      %120 : Float(1, 320, strides=[320, 1], requires_grad=0, device=cuda:0),
      %121 : Long(1, strides=[1], requires_grad=0, device=cpu),
      %122 : Long(1, strides=[1], requires_grad=0, device=cpu)):
  %11 : Float(1, 32, 496, strides=[15872, 

/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:2119: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn("Exporting a model to ONNX with a batch_size other than 1, " +


### load the onnx model

In [11]:
import onnx

# Load the ONNX model
model = onnx.load(os.path.join(onnx_output_folder, "fcrnn.onnx"))

# Check that the IR is well formed
onnx.checker.check_model(model)

### Make some test inference with onnx

In [12]:
import onnxruntime as ort

ort_session = ort.InferenceSession(os.path.join(onnx_output_folder, "fcrnn.onnx"))

outputs = ort_session.run(
    None,
    {'input': np.random.randn(1, 201, 498).astype(np.float32)}
)
outputs

2022-03-10 14:51:47.906341705 [W:onnxruntime:, graph.cc:1237 Graph] Initializer 0.weight appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2022-03-10 14:51:47.906921146 [W:onnxruntime:, graph.cc:1237 Graph] Initializer 0.bias appears in graph inputs and will not be treated as constant value/weight. This may prevent some of the graph optimizations, like const folding. Move it out of graph inputs if there is no need to override it, by either re-generating the model with latest exporter/converter or with the tool onnxruntime/tools/python/remove_initializer_from_input.py.
2022-03-10 14:51:47.906930713 [W:onnxruntime:, graph.cc:1237 Graph] Initializer 3.weight appears in graph inputs and will not 

[array([[[ 0.00070733, -0.01375907, -0.00012339, ...,  0.0065646 ,
           0.00439894, -0.01056484],
         [-0.006847  , -0.01803691, -0.02265233, ..., -0.01419169,
           0.0101109 , -0.02161603],
         [ 0.00352269, -0.037413  , -0.00238217, ...,  0.01082113,
           0.01381979, -0.00142794],
         ...,
         [-0.0111691 , -0.01851932, -0.01398994, ..., -0.01366919,
           0.00501725, -0.02036662],
         [-0.01949888, -0.0196784 , -0.00619684, ..., -0.00768538,
           0.00257744, -0.00301417],
         [-0.01891368, -0.01620599, -0.00448724, ...,  0.00270507,
           0.00804203, -0.00138754]]], dtype=float32),
 array([[[ 0.00070733, -0.01375907, -0.00012339, ...,  0.0065646 ,
           0.00439894, -0.01056484],
         [-0.006847  , -0.01803691, -0.02265233, ..., -0.01419169,
           0.0101109 , -0.02161603],
         [ 0.00352269, -0.037413  , -0.00238217, ...,  0.01082113,
           0.01381979, -0.00142794],
         ...,
         [-0.01116

### ONNX to tf

In [13]:
from onnx_tf.backend import prepare

tf_output_folder = "best_model_conversion/tf"

tf_rep = prepare(model)
tf_rep.export_graph(os.path.join(tf_output_folder, "fcrnn"))

2022-03-10 14:51:48.593500: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py:458: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  return f(*args, **kwargs)
/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:984: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._kernel = self.add_variable(
/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:993: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._bias = self.add_variable(
/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packag

/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/keras/legacy_tf_layers/base.py:573: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  _add_elements_to_collection(self.updates, tf.compat.v1.GraphKeys.UPDATE_OPS)


INFO:tensorflow:Assets written to: best_model_conversion/tf/fcrnn/assets


INFO:tensorflow:Assets written to: best_model_conversion/tf/fcrnn/assets


### TF to tflite

Salade de tensorflow

In [14]:
import tensorflow as tf

# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:
#     try:
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
#     except RuntimeError as e:
#         print(e)

In [15]:
# model = tf.saved_model.load(os.path.join(tf_output_folder, "fcrnn"))
model = tf.saved_model.load("best_model_conversion/tf/fcrnn")
model.trainable = False

input_tensor = tf.random.uniform([1, 201, 498])
out = model(**{'input': input_tensor})

2022-03-10 14:51:54.965842: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/assert_equal_1/Assert/AssertGuard/branch_executed/_24


InvalidArgumentError: Graph execution error:

Detected at node 'LSTM_3e64b8c7/rnn/while/rnn/multi_rnn_cell/cell_0/lstm_cell/BiasAdd' defined at (most recent call last):
    File "/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/mus5900/anaconda3/envs/series_forcasting/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/mus5900/anaconda3/envs/series_for